## Data-preparatie

Dit notebook behandelt de data-preparatie en geeft antwoord op deelvraag 1:

*"Hoe kan cellijn-data (mutaties, methylatie, drug-respons) worden verzameld, opgeschoond en geschikt gemaakt voor machine learning?"*

Voor dit onderzoek worden drie afzonderlijke datasets gebruikt die afkomstig zijn van het CCLE-project (Cancer Cell Line Encyclopedia). De data beschrijven verschillende moleculaire kenmerken van kankercellijnen:

- OmicsSomaticMutations.csv - bevat de somatische mutatieprofielen van de cellijnen.
- CCLE_RRBS_TSS1kb_20181022.txt - bevat DNA-methylatiegegevens gemeten rond de Transcription Start Site (TSS, ±1 kb). Dit gebied omvat zowel promoterregio's als het eerste exon, die belangrijk zijn voor genregulatie.
- Drug_sensitivity_AUC_(PRISM_Repurposing_Secondary_Screen)_subsetted.csv - bevat therapieresponsgegevens (AUC-waarden) die aangeven hoe gevoelig een cellijn is voor een bepaald medicijn.

Het uiteindelijke doel is om een dataset te creëren waarin elke longkankercellijn zowel moleculaire kenmerken (mutaties en methylatie) als een responswaarde (AUC) bevat, zodat deze kan worden gebruikt als input voor supervised machine learning.

**xxx** toelichting CpG sites en avg coverage (kwaliteitsmaat) xxx:
CpG_sites_hg19 = genomische voordinaten van CpG-sites in het Hg19-genoom
avg_coverage = maat voor de betrouwbaarheid van de methylatie-metingen op dat locus

TSS=Transcription Start Site, het beginpunt van een gen waar transcriptie start.
±1kb betekent dat de gemeten regio 1 kilobase (1000bp) upstream en downstream van de TSS ligt
dus direct rond het begin van het gen, inclusief een stukje voor (promotorgebied) en erna (eerste exon/regio vlakbij TSS)

- Upstream van TSS (~1000 bp): dit is meestal de promoter. Methylatie hier kan de transcriptie van het gen onderdrukken.
- Downstream van TSS (~1000 bp): dit is vaak het 5’ einde van het gen, inclusief exon 1 of de 5’ UTR. Methylatie hier kan ook effect hebben op genexpressie. methylatie in deze regio kan het initiatiefase van transcriptie veranderen, waardoor het gen minder of anders wordt afgelezen

Nadenken over avg_coverage. minimum instellen. te lagen waarden weglaten omdat de methylatiepercentages dan ruis bevatten (in de literatuur 10>)
laat ook zien want de impact is van die drempel.


#### Stap 1. Data verkennen

Doel: inzicht krijgen in de structuur van elk bestand voor het filteren en samenvoegen.

Het is belangrijk om de kolomnamen, rijenstructuur en datatypes te begrijpen. Zo krijg je inzicht in welke kolommen relevant zijn (bijv. cell_line_id, gene, AUC, avg_coverage) en kun je bepalen hoe de datasets aan elkaar gekoppeld kunnen worden.

stappen:
Controleer het aantal rijen/kolommen.
Bekijk een paar voorbeeldrijen om de structuur te snappen.
Noteer de naam van de kolom die de cellijn identificeert (ID).
Kijk of de kolomnamen consistent zijn tussen bestanden.
Kijk hoeveel unieke cellijnen er per bestand zijn.))


In [11]:
import pandas as pd

# aanmaken variabele met de bestandpad
mutatie_file = "../data/raw/OmicsSomaticMutations.csv"

# inlezen 
mutatie_df = pd.read_csv(mutatie_file) # csv, geen tabs

# verkenning
mutatie_df.info()
print(mutatie_df.head()) 


/var/folders/hf/q9gpk8mn1l34cwvqff64g18c0000gn/T/ipykernel_14934/1651157059.py:7: DtypeWarning: Columns (22,50,56,57,58,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  mutatie_df = pd.read_csv(mutatie_file) # csv, geen tabs


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 751310 entries, 0 to 751309
Data columns (total 70 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Chrom                            751310 non-null  object 
 1   Pos                              751310 non-null  int64  
 2   Ref                              751310 non-null  object 
 3   Alt                              751310 non-null  object 
 4   AF                               751310 non-null  float64
 5   DP                               751310 non-null  int64  
 6   RefCount                         751310 non-null  int64  
 7   AltCount                         751310 non-null  int64  
 8   GT                               751310 non-null  object 
 9   PS                               44923 non-null   float64
 10  VariantType                      751310 non-null  object 
 11  VariantInfo                      751310 non-null  object 
 12  DN

De mutatie-dataset bevat 751310 rijen en 70 kolommen en bestaat uit de volgende datatypen:

- object: tekst/categorieën (bijvoorbeeld genen en mutatietypes)
- float64: decimale waarden
- bool: waar/onwaar waarden (True/False)
- int64: hele getallen (bijvoorbeeld aantallen mutaties)

Belangrijk voor dit onderzoek:
ModelID: koppeling tussen datasets
HugoSymbol/EnsemblGeneID: gen waarin mutatie voorkomt
VariantType: type mutatie (feature voor ML)
DNAChange/ProteinChange: optioneel voor interpretatie
AF, RefCount, AltCount: betrouwbaarheid / filteren
LikelyLoF, OncogeneHighImpact, TumorSuppressorHighImpact: impact op functie/belangrijke features
Sift, Polyphen, RevelScore: schadelijkheidsvoorspelling
PrimaryDisease/Tissue: filteren op longkankercellijnen

In [16]:
# aanmaken variabele met de bestandpad
methylatie_file = "../data/raw/CCLE_RRBS_TSS1kb_20181022.txt"

# inlezen
methylatie_df = pd.read_csv(methylatie_file, sep='\t') # txt, tab seperated

# verkenning
print("Aantal rijen en kolommen:", methylatie_df.shape) # globale info

# eerste en laatste 5 kolmmen bekijken
print("Eerste 5 kolommen:", methylatie_df.columns[:10].tolist())
print("Laatste 5 kolommen:", methylatie_df.columns[-10:].tolist())

# datatypes en missende waarden per kolom bekijken
kolom_info_methylatie = pd.DataFrame({ # aanmaken dataframe (met 3 kolommen)
    'kolom': methylatie_df.columns,
    'dtype': [methylatie_df[col].dtype for col in methylatie_df.columns],
    'n_missing': [methylatie_df[col].isna().sum() for col in methylatie_df.columns] # bepaald totaal aantal missende waarden voor elke kolom
})

# eerste 10 en laatste 10 kolommen tonen als voorbeeld
print(kolom_info_methylatie.head(10)) # eerste 10 kolommen
print(kolom_info_methylatie.tail(10)) # laatste 10 kolommen

/var/folders/hf/q9gpk8mn1l34cwvqff64g18c0000gn/T/ipykernel_14934/1102694824.py:5: DtypeWarning: Columns (2,618) have mixed types. Specify dtype option on import or set low_memory=False.
  methylatie_df = pd.read_csv(methylatie_file, sep='\t') # txt, tab seperated


Aantal rijen en kolommen: (21338, 846)
Eerste 5 kolommen: ['locus_id', 'CpG_sites_hg19', 'avg_coverage', 'DMS53_LUNG', 'SW1116_LARGE_INTESTINE', 'P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'UMUC3_URINARY_TRACT', 'HOS_BONE', 'AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE']
Laatste 5 kolommen: ['JHUEM7_ENDOMETRIUM', 'OE21_OESOPHAGUS', 'MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'HOP62_LUNG', 'EKVX_LUNG', 'UO31_KIDNEY', 'SF268_CENTRAL_NERVOUS_SYSTEM', 'SF539_CENTRAL_NERVOUS_SYSTEM', 'SNB75_CENTRAL_NERVOUS_SYSTEM', 'HOP92_LUNG']
                                       kolom   dtype  n_missing
0                                   locus_id  object          0
1                             CpG_sites_hg19  object          1
2                               avg_coverage  object          0
3                                 DMS53_LUNG  object          0
4                     SW1116_LARGE_INTESTINE  object          0
5   P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE 

De methylatie-dataset bevat 21338 rijen en 846 kolommen. Elke rij vertegenwoordigt een specifieke CpG-locus, een locatie op het DNA waar methylatie kan optreden.

De eerste kolommen bevatten metadata:
- `locus_id`: unieke identifier van de CpG-locus
- `CpG_sites_hg19`:genoompositie van de locus
- `avg_coverage`: gemiddelde meetdiepte, een maat voor de betrouwbaarheid van de meting

Alle overige kolommen (vanaf kolom 4) vertegenwoordigen de cellijnen. Wat me opvalt is dat dit bestand geen ACH- of ModelID bevat. In plaats daarvan worden cellijnen geidentificeerd op basis van hun naam,  waarbij de kolomtitel zowel de cellijnnam als het weefseltype bevat  (bijvoorbeeld `DMS53_LUNG`).

**Let op!** Om de drie datasets samen te voegen zal een mapping moeten worden gemaakt van deze kolomnamen naar de bijbehorende ACH_ID. Dit zal worden uitgevoerd in stap 5 (samenvoegen).

In [40]:
# aanmaken variabele met de bestandpad
response_file = "../data/raw/Drug_sensitivity_AUC_(PRISM_Repurposing_Secondary_Screen)_subsetted.csv"
# inlezen
response_df = pd.read_csv(response_file, sep=',') # comma seperated

# datatypes en missende waarden per kolom bekijken
kolom_info_response = pd.DataFrame({
    'kolom': response_df.columns,
    'dtype': [response_df[col].dtype for col in response_df.columns],
    'n_missing': [response_df[col].isna().sum() for col in response_df.columns]
})

# eerste 20 kolommen tonen als voorbeeld
print(kolom_info_response.head(10))
print(kolom_info_response.tail(10))


                                               kolom    dtype  n_missing
0                                          depmap_id   object          0
1                             cell_line_display_name   object          0
2                                          lineage_1   object          0
3                                          lineage_2   object          0
4                                          lineage_3   object          0
5                                          lineage_6   object        436
6                                          lineage_4  float64        480
7          8-BROMO-CGMP (BRD:BRD-A00077618-236-07-6)  float64        370
8          NORETYNODREL (BRD:BRD-A00758722-001-04-9)  float64         98
9  PREDNISOLONE-ACETATE (BRD:BRD-A01643550-001-04-9)  float64        432
                                                  kolom    dtype  n_missing
1447            LORLATINIB (BRD:BRD-K99879819-001-02-1)  float64        326
1448       HEXYLRESORCINOL (BRD:BRD-K99946902

In [46]:
# aanmaken variabele met de bestandpad
response_file = "../data/raw/Drug_sensitivity_AUC_(PRISM_Repurposing_Secondary_Screen)_subsetted.csv"
# inlezen
response_df = pd.read_csv(response_file, sep=',') # comma seperated

# verkenning
print("Aantal rijen en kolommen:", response_df.shape) # geeft het aantal rijen en kolommen

# datatypes en missende waarden per kolom bekijken
kolom_info_response = pd.DataFrame({
    'kolom': response_df.columns,
    'dtype': [response_df[col].dtype for col in response_df.columns],
    'n_missing': [response_df[col].isna().sum() for col in response_df.columns]
})

# eerste 20 kolommen tonen als voorbeeld
print(kolom_info_response.head(10))
print(kolom_info_response.tail(10))

print(response_df.iloc[:5,:10])

Aantal rijen en kolommen: (480, 1457)
                                               kolom    dtype  n_missing
0                                          depmap_id   object          0
1                             cell_line_display_name   object          0
2                                          lineage_1   object          0
3                                          lineage_2   object          0
4                                          lineage_3   object          0
5                                          lineage_6   object        436
6                                          lineage_4  float64        480
7          8-BROMO-CGMP (BRD:BRD-A00077618-236-07-6)  float64        370
8          NORETYNODREL (BRD:BRD-A00758722-001-04-9)  float64         98
9  PREDNISOLONE-ACETATE (BRD:BRD-A01643550-001-04-9)  float64        432
                                                  kolom    dtype  n_missing
1447            LORLATINIB (BRD:BRD-K99879819-001-02-1)  float64        326
1448   

De response-dataset bevat 480 rijen, waarbij elke rij een unieke cellijn representeert, en 1457 kolommen. 
De eerste kolommen bevatten meta-data over de cellijnen:

- depmap_id: unieke ID voor elke cellijn (bijvoorbeeld ACH-000973)
- cell_line_display_name: leesbare naam van de cellijn (bijvoorbeeld 639V of BECKER)
- lineage_1 t/m lineage_6: beschrijving van de biologische afkomst van de cellijn, van breed tot specifiek.

Vanaf kolom 8 bevatten de overige kolommen de drug-response waarden voor verschillende geneesmiddelen, weergegeven als numerieke scores (AUC). Elke cellijn is getest voor meerdere geneesmiddelen, waarbij sommige waarden ontbreken (NaN).


#### Stap 2. Filteren op relevante data

Doel: alleen de biologische en analyse relevante mutaties behouden, zodat de dataset overzichtelijk blijft en rekentijd wordt bespaard.

**Mutatiebestand**

Acties:
1. filteren op cellijntype: alleen cellijnen van het weefseltype "lung" worden behouden.
2. filteren op relevante mutaties: niet alle mutaties zijn biologisch significant (veel varianten zijn neutraal of technisch artefact). Daarom worden alleen functionele mutaties behouden op basis van de volgende kolommen:
   - LiklyLoF (verlies-van-functie mutaties)
   - OncogeneHighImpact (activerende mutaties in oncogenen)
   - TumorSuppressorHighImpact (verlies-van-remfunctie in tumorsuppressorgen)
   - Hotspot (bekende activerende hotspot-mutaties)
   - HessDriver (mutaties geidentificeerd als drivers)
   Een mutatie wordt behouden als één of meer van deze criteria waar zijn.
3. transformatie naar binaire matrix: voor elke cellijn (ModelID) wordt per gen (HugoSymbol) gekeken of er een relevante mutate aanwezig is. Wanneer een mutatie wordt gevonden dan krijgt deze waarde=1, anders waarde=0. Dit resulteert in een binaire matrix waarin rijen en cellijnen representeren en kolommen genen.


In [55]:
# Filter alleen biologisch relevante mutaties
filtered = mutatie_df[
    mutatie_df['LikelyLoF'] |
    mutatie_df['OncogeneHighImpact'] |
    mutatie_df['TumorSuppressorHighImpact'] |
    mutatie_df['Hotspot'] |
    mutatie_df['HessDriver']
]

# Maak binaire matrix per gen per celijn
mut_matrix = (
    filtered
    .assign(mutated=1)
    .pivot_table(index='ModelID', columns='HugoSymbol', values='mutated', fill_value=0)
    .astype(int) 
)

print(mut_matrix.shape)
print(mut_matrix.head)


(1939, 18598)
<bound method NDFrame.head of HugoSymbol  A1BG  A1CF  A2M  A2ML1  A3GALT2  A4GALT  A4GNT  AAAS  AACS  AADAC  \
ModelID                                                                         
ACH-000001     0     0    0      0        0       0      0     1     0      0   
ACH-000002     0     0    0      0        0       0      0     0     0      0   
ACH-000004     0     0    0      0        0       0      0     0     0      0   
ACH-000005     0     0    0      0        0       0      0     0     0      0   
ACH-000006     0     0    0      0        0       0      0     0     0      0   
...          ...   ...  ...    ...      ...     ...    ...   ...   ...    ...   
ACH-003473     0     0    0      0        0       0      0     0     0      0   
ACH-003474     0     0    0      0        0       0      0     0     0      0   
ACH-003475     0     0    0      0        0       0      0     0     0      0   
ACH-003476     0     0    0      0        0       0      0     0 

Output: 
- elke rij is een longkankercellijn
- elke kolom is een gen
- een waarde van 1 betekent dat er in die cellijn minstens één functionele mutatie aanwezig is in dat gen
- een waarde van 0 betekent dat er in die cellijn geen relevante mutatie aanwezig is in dat gen

Deze gefilterde en geformatteerde matrix is nu klaar om te geombineerd te worden met de andere datasets (methylatie, drug-response)

#### Stap 3. Opschonen

Doel: ervoor zorgen dat alle bestanden dezelfde structuur en sleutel-IDs gebruiken.

De kolomnamen moeten worden gestandaardiseerd. Dit houd in dat alledrie de bestanden dezelfde sleutel-IDs moeten krijgen: cell_line_id. Ook moeten dubbele rijen of niet-unieke cellijnen moeten worden verwijderd. Er moet gecontroleerd worden op ontbrekende waarden (NaN) in de belangrijke kolommen. Dit is belangrijk om fouten te voorkomen bij het samenvoegen.

**xxx** avc_covverage in deze stap? > Waarden lager dan 10 worden verwijderd, omdat deze metingen onvoldoende dekking hebben om betrouwbare methylatiegegevens te geven. Dit voorkomt dat technische ruis invloed heeft op het model.

Binaire mutatiematrix per cellijn
In deze stap wordt een binaire mutatiematrix aangemaakt op a

#### Stap 4. Transformatie van features

Doel: 
- mutatiegegevens omzetten naar binaire variabelen (0=mutatie, 1=mutatie).
- methylatiegegevens reduceren/aggregeren (gemiddelde pakken?)


Machine-learningmodellen werken met numerieke inputs; binaire encoding wordt veelgebruikt.

Methylatie heeft vaak duizenden features wat kan leiden tot overfitting? **meer uitleg nodig, bron**

Wanneer er meerdere AUC-metingen zijn per cellijn moet het gemiddelde worden berekend.

#### Stap 5. Samenvoegen 

Doel: het creëren van één tabel waarin elke rij één cellijn is, met alle relevante features en bijbehorende responswaarde.

De drie datasets worden samengevoegd op de gemeenschappelijke sleutel cell_line_id. 

Er wordt gebruik gemaakt van een **inner join** om ontbrekende waarden in de uiteindelijke tabel te voorkomen. Alleen cellijnen waarvan zowel moleculaire data (mutaties, methylatie) als responsdata beschikbaar zijn, worden behouden. Dit is een belangrijke eis voor supervised learning.

check ook hoeveel cellijnen verloren gaan door die joing.


#### Stap 6. Controle op ontbrekende waarden

Doel: controleren of de uiteindelijke dataset compleet genoeg is.

Cellijnen met onbrekende AUC moeten worden verwijderd omdat machine-learningalgoritmen niet kunnen omgaan met ontbrekende waarden. 



#### Stap 7. Preprocessing voor machine learning

Doel: de data omzetten naar numerieke vorm, schalen en voorbereiden voor modellering.

**hier start scikit-learn** Pandas is vooral geschikt voor het inlezen, bewerken en samenvoegen van dataframes. Scikit-learn is gespecialiseerd in preprocessing. 

**xxx** verwijs naar scikit-learn tutorial?




